In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
DATA_DIR = "drive/MyDrive/AI_X/"

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils.vis_utils import plot_model

from pathlib import Path
import pickle, PIL, io, os
import zipfile
from os.path import exists
from tqdm import tqdm

In [4]:
image_df = pd.read_csv(DATA_DIR+"csv/image_dir_2_.csv")

In [5]:
le = preprocessing.LabelEncoder()
le.fit(image_df.Label)
le.classes_

array(['가구끄는소리', '강아지짓는소리', '골프연습장의타구음',
       '골프퍼팅(골굴리는소리)', '등하원아이들떠드는소리',
       '런닝머신에서뛰는소리', '망치질소리', '문여닫는소리',
       '바이올린연주소리', '샤워할때물소리', '세탁기소리',
       '소리없음', '식기세척기소리',
       '심야에이륜차가빠르게주행하는소리',
       '심야에자동차가빠르게주행하는소리',
       '에어컨실외기의가동소리', '진공청소기소리',
       '피아노연주소리', '화장실물내리는소리', '발걸음소리'], dtype=object)

In [6]:
train_labels = np.array(le.transform(image_df.Label))
train_labels

array([ 1,  1,  1, ..., 11, 11, 11])

In [7]:
le.inverse_transform(train_labels)

array(['강아지짓는소리', '강아지짓는소리', '강아지짓는소리', ...,
       '소리없음', '소리없음', '소리없음'], dtype=object)

In [8]:
TRAIN_IMAGES_FILE = os.path.join(DATA_DIR + "pkl/null_add/RGB/150_50/train_images.pkl")

if exists(TRAIN_IMAGES_FILE):
    with open(TRAIN_IMAGES_FILE, 'rb') as f:
        train_images = pickle.load(f)
else:
    train_images = load_images(image_df.Filepath)
    with open(TRAIN_IMAGES_FILE, 'wb') as f:
        pickle.dump(train_images, f, pickle.HIGHEST_PROTOCOL)

In [9]:
train_images.shape

(22275, 50, 150, 3)

In [10]:
x = train_images
y = train_labels

X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=train_labels)
X_train.shape, X_valid.shape

((17820, 50, 150, 3), (4455, 50, 150, 3))

In [11]:
X_train = np.array(X_train) / 255
X_valid = np.array(X_valid) / 255

In [12]:
y_train = np.array(y_train)
y_valid = np.array(y_valid)

In [13]:
from tensorflow.keras.layers import BatchNormalization, MaxPool2D, Conv2D

In [14]:
model2 = tf.keras.Sequential()
model2.add(Conv2D(32, (3,3), strides=1, padding='same', activation='relu', input_shape=(50,150,3)))
model2.add(BatchNormalization())
model2.add(MaxPool2D((3,3), strides=2 , padding='same'))
model2.add(Conv2D(64, (3,3), strides=1, padding='same', activation='relu'))
model2.add(Dropout(0.1))
model2.add(BatchNormalization())
model2.add(MaxPool2D((3,3), strides=2, padding='same'))
model2.add(Conv2D(64, (3,3), strides=1, padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPool2D((3,3), strides=2, padding='same'))
model2.add(Conv2D(128, (3,3), strides=1, padding='same', activation='relu'))
model2.add(Dropout(0.2))
model2.add(BatchNormalization())
model2.add(MaxPool2D((3,3), strides=2, padding='same'))
model2.add(Conv2D(256, (3,3), strides=1, padding='same', activation='relu'))
model2.add(Dropout(0.2))
model2.add(BatchNormalization())
model2.add(MaxPool2D((3,3), strides=2, padding='same'))
model2.add(Flatten())
model2.add(Dense(units=128, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(units=(len(le.classes_)), activation='softmax'))


model2.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy']) #optimizer : rmsprop, adam
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 150, 32)       896       
                                                                 
 batch_normalization (BatchN  (None, 50, 150, 32)      128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 75, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 25, 75, 64)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 25, 75, 64)       2

In [15]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1,factor=0.2, min_lr=0.000001)

In [16]:
model2.fit(
    X_train,
    y_train,
    # Use your own validation set.
    validation_data=(X_valid, y_valid),
    epochs=100,
    callbacks = [early_stopping, learning_rate_reduction]
)
print("Accuracy:", model2.evaluate(X_valid, y_valid)[1])

Epoch 1/100
557/557 [==============================] - 13s 18ms/step - loss: 1.0878 - accuracy: 0.6488 - val_loss: 2.5863 - val_accuracy: 0.3582 - lr: 0.0010
Epoch 2/100
557/557 [==============================] - 9s 16ms/step - loss: 0.5891 - accuracy: 0.8001 - val_loss: 2.7428 - val_accuracy: 0.5199 - lr: 0.0010
Epoch 3/100
557/557 [==============================] - 9s 16ms/step - loss: 0.4447 - accuracy: 0.8567 - val_loss: 0.7533 - val_accuracy: 0.7526 - lr: 0.0010
Epoch 4/100
557/557 [==============================] - 9s 16ms/step - loss: 0.3665 - accuracy: 0.8788 - val_loss: 0.8711 - val_accuracy: 0.7596 - lr: 0.0010
Epoch 5/100
557/557 [==============================] - 9s 16ms/step - loss: 0.3340 - accuracy: 0.8900 - val_loss: 1.3320 - val_accuracy: 0.6559 - lr: 0.0010
Epoch 6/100
557/557 [==============================] - 9s 16ms/step - loss: 0.2905 - accuracy: 0.9053 - val_loss: 0.7871 - val_accuracy: 0.7636 - lr: 0.0010
Epoch 7/100
557/557 [==============================] - 9s

In [17]:
y_pred = le.inverse_transform(np.array([np.argmax(pred) for pred in model2.predict(X_valid)]))
y_test = le.inverse_transform(y_valid)

140/140 [==============================] - 1s 4ms/step


In [18]:
print(classification_report(y_test, y_pred))

                                      precision    recall  f1-score   support

                       가구끄는소리       0.99      0.85      0.91       208
                   강아지짓는소리       1.00      0.99      0.99       212
             골프연습장의타구음       1.00      0.98      0.99       209
         골프퍼팅(골굴리는소리)       0.99      1.00      1.00       213
          등하원아이들떠드는소리       0.99      0.99      0.99       200
            런닝머신에서뛰는소리       1.00      1.00      1.00       213
                        망치질소리       0.98      0.96      0.97       224
                     문여닫는소리       0.95      0.98      0.96       216
                 바이올린연주소리       1.00      1.00      1.00       225
                    샤워할때물소리       0.99      0.99      0.99       211
                         세탁기소리       0.91      0.95      0.93       425
                          소리없음       1.00      0.50      

In [19]:
model2.save(DATA_DIR+"/h5/null/model_CNN_2_97_best.h5")